Consider an \( n \)-sided regular polygon with charges of equal magnitude placed at each vertex.

1. Create a program that computes the position of each charge in the \( x \)-\( y \) plane.
2. Plot the electric potential due to the collection of these charges.


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive, FloatSlider, IntSlider
from IPython.display import display

def potential_plot(elev, azim, q, s, n, y_slice):
    # Set up the figure with two subplots
    plt.rcParams['figure.figsize'] = [15, 6]
    fig = plt.figure()
    
    # Create 3D subplot
    ax1 = fig.add_subplot(121, projection='3d')
    
    # Create 2D subplot
    ax2 = fig.add_subplot(122)
    
    # Original potential calculation
    theta0 = np.pi/n
    R = s/(2 * np.sin(theta0))
    
    def potential(q, x, y, x0, y0):
        k = 9e9
        r = np.sqrt((x - x0) ** 2 + (y - y0) ** 2 + 1e-9)
        return k * q / r
    
    # Creating 2D grid
    X = np.linspace(-5, 5, 500)
    Y = np.linspace(-5, 5, 500)
    x, y = np.meshgrid(X, Y)
    
    # Calculate vertices positions
    Xm = np.array([R * np.cos(2 * m * theta0) for m in range(n)])
    Ym = np.array([R * np.sin(2 * m * theta0) for m in range(n)])
    
    # Calculate net potential
    Vnet = sum(potential(q, x, y, xm, ym) for xm, ym in zip(Xm, Ym))
    
    epsilon = 1e-7
    Vplot = np.sign(Vnet) * np.log10(np.abs(Vnet) + epsilon)
    
    # Plot 3D surface
    surf = ax1.plot_surface(x, y, Vplot, cmap='magma', edgecolor='none',
                          antialiased=True, rcount=100, ccount=100, alpha=0.9)
    
    # Add zero potential contours
    ax1.contour(x, y, Vplot, levels=[0], colors='green', linewidths=2, zdir='z', offset=Vnet.min())
    ax1.contour3D(x, y, Vplot, levels=[0], colors='#00FFFF', linewidths=2)
    
    # Set 3D plot properties
    fig.colorbar(surf, ax=ax1, shrink=0.7, aspect=10, pad=0.1, label=r'$\log(V)$')
    ax1.set_xlabel('x', labelpad=10)
    ax1.set_ylabel('y', labelpad=10)
    ax1.set_zlabel(r'$\log(V)$', labelpad=10)
    ax1.set_title('3D Potential', pad=20, y=1.05)
    ax1.view_init(elev=elev, azim=azim)
    ax1.dist = 8
    ax1.set_box_aspect([1, 1, 0.8])
    ax1.set_xlim(-6, 6)
    ax1.set_ylim(-6, 6)

    # Create slice plane
    slice_x = np.linspace(-6, 6, 100)
    slice_z = np.linspace(Vplot.min(), Vplot.max(), 100)
    slice_X, slice_Z = np.meshgrid(slice_x, slice_z)
    slice_Y = np.full_like(slice_X, y_slice)  # Y-value from slider
    
    # Plot the transparent plane
    ax1.plot_surface(slice_X, slice_Y, slice_Z, 
                    alpha=0.2,  # Transparency
                    color='yellow',  # Plane color
                    shade=False)  # Disable shading for clearer view
    
    # Optional: Add a line at the intersection
    y_idx = np.abs(Y - y_slice).argmin() # Find the index closest to our y_slice value
    ax1.plot(X, [y_slice]*len(X), Vplot[y_idx, :], 
             color='yellow',  # Line color
             linewidth=2)  # Line thickness
    
    # Plot the slice
    ax2.plot(X, Vplot[y_idx, :], 'b-', linewidth=2)
    ax2.grid(True)
    ax2.set_xlabel('x')
    ax2.set_ylabel(r'$\log(V)$')
    ax2.set_title(f'Potential at y = {y_slice:.2f}')
    ax2.set_xlim(-6, 6)
    
    # Add a horizontal line at V=0
    ax2.axhline(y=0, color='r', linestyle='--', alpha=0.5)
    
    # Add vertical lines where charges are located
    for xm, ym in zip(Xm, Ym):
        if abs(ym - y_slice) < 0.2:  # Only show charges near the slice
            ax2.axvline(x=xm, color='g', linestyle=':', alpha=0.5)
    
    plt.tight_layout()
    plt.show()

# Create interactive plot with added y_slice slider
interactive_plot = interactive(potential_plot,
                             elev=IntSlider(min=-90, max=90, step=5, value=25),
                             azim=IntSlider(min=0, max=360, step=5, value=45),
                             q=FloatSlider(min=-5, max=5, step=0.1, value=1, description='q'),
                             s=FloatSlider(min=0, max=5, step=0.1, value=1, description='s'),
                             n=IntSlider(min=3, max=100, step=1, value=3, description='n'),
                             y_slice=FloatSlider(min=-5, max=5, step=0.1, value=0, description='y-slice'))

display(interactive_plot)

interactive(children=(IntSlider(value=25, description='elev', max=90, min=-90, step=5), IntSlider(value=45, de…